In [ ]:
#!pip install pandas transformers torch

Special install of pytorch for apple sillicon

In [ ]:
#!pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

In [7]:
# Import necessary libraries
import pandas as pd
from transformers import pipeline
import utils

In [8]:
# Step 1: Load and Parse the possible classes
# =================================

# Load the CSV file with categories and identifiers
categories = pd.read_csv('hierarchy/FoR_code_hierarchical_4_digit.csv')
# Build the Hierarchical Structure with Identifiers
hierarchy_with_ids = utils.build_hierarchy_with_ids(categories)

In [10]:
# Step 2: Initialize the Hugging Face Model
# ==========================================

# Load the Hugging Face model for classification
classifier = pipeline("zero-shot-classification",
                      #model="facebook/bart-large-mnli")
                      model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0",device = "mps")


In [ ]:
# Step 3: Load the CSV file with papers to be identified
papers = pd.read_csv('sample_data/dev_sample.csv')

In [11]:
# #title = "From Maladaptation to Competition to Cooperation in the Evolution of Musical Behaviour"
# #journal = "Musicae Scientiae"
# #title = "Protruding masticatory (superfast) myosin heads from staggered thick filaments of dog jaw muscle revealed by X-ray diffraction"
# #journal = "Journal of Biochemistry"
# title = "Characterising infant and young child feeding practices and the consumption of poultry products in rural Tanzania: A mixed methods approach"
# journal = "Maternal and Child Nutrition"
# abstract = "Suboptimal breastfeeding practices, early initiation of complementary feeding, and monotonous cereal-based diets have been implicated as contributors to continuing high rates of child undernutrition in sub-Saharan Africa. Nutrition-sensitive interventions, including agricultural programs that increase access to nutrient-rich vegetables, legumes, and animal-source foods, have the potential to achieve sustainable improvements in children's diets. In the quest to evaluate the efficacy of such programs in improving growth and development in the first 2 years of life, there is a role for mixed methods research to better understand existing infant and young child feeding practices. This analysis forms part of a longitudinal study assessing the impact of improvements to poultry health and crop production on diets and growth of 503 randomly selected children from eight rural communities in Manyoni District in central Tanzania. Using an explanatory sequential design, the quantitative phase of data collection was conducted between May 2014 and May 2016, comprising six monthly structured questionnaires, four monthly household-level documentation of chicken and egg consumption, and fortnightly records of children's breastfeeding status. The subsequent qualitative phase involved in-depth interviews with a subset of 39 mothers in October 2016. Breastfeeding was almost universal (96.8%) and of long duration (mean = 21.7 months, SD = 3.6), but early initiation of complementary feeding was also common (74.4%; mean = 4.0 months, SD = 1.8), overwhelmingly driven by maternal perceptions of insufficient milk supply (95.0%). Chicken and eggs were infrequently eaten, but close associations between maternal and child consumption patterns (p <.001) suggest the potential for strategies that increase household-level consumption to bring nutritional benefits to young children."

# def tst_fnc():
#     #query = "Journal: "+journal+", Title: "+physics_paper
#     #query = "Journal: "+journal+", Title: "+title
#     query = title
#     #query = title + ". Abstract: "+ abstract
#     #query = "Journal: "+journal+", Title: "+title+", Abstract: "+ abstract
#     #query = "Abstract: "+ abstract+", Title: "+title+", Journal: "+journal
#  #   return classify_text_with_ids(query, classifier, hierarchy_with_ids,threshold=0.3,n_max=10)
#     return classify_text_with_ids(query, classifier, hierarchy_with_ids,threshold=0.7)

# #elapsed_time = timeit.timeit(tst_fnc, number=1)
# #print(f"Elapsed time: {elapsed_time} seconds")
# result = tst_fnc()

# flattened_list = [item for sublist in result for item in sublist]

# # Display the results
# print("Final Classification Results:")
# print(flattened_list)
# #result2 = [expression(item) for item in result]
# #print(list.flatten(result))

# #for path, prob, hierarchy_path in result:
#    print(f"Path: {path}, Probability: {prob:.4f}\n")

In [ ]:
# Define the template with placeholders for the columns
query_template = "Title: {Publication Title}"

In [14]:
# Call the function, passing the template as an argument
output = utils.classify_papers_with_template(
    papers, 
    classifier, 
    hierarchy_with_ids, 
    template=query_template, 
    threshold=0.7, 
    n_max=3
)

In [ ]:
# Display the first few rows of the DataFrame
#print(output.head())

In [15]:
# Define the match configuration with match types and number of digits
match_config = {
    'Division': 2,
    'Group': 4
}

# Apply the function to the DataFrame
matched = utils.apply_code_matches(output, match_config)

In [16]:
# Write out the csv
matched.to_csv('./output/dev_output.csv', index=False)

In [ ]:
# Calculate performance metrics
